# 의사결정나무 연습

##### criterion 파라미터 값을 변경하는데에 있어 조금 더 이해도를 높이기 위해 다른 파일로 연습을 해보았습니다.
---

#####  

# Dataset import
## 데이터 불러오기

In [1]:
# import libraries

import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import tree

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

# raw_wine = datasets.load_wine()

In [2]:
# 데이터 셋 내 피처 살펴보기
raw_melb = pd.read_csv("/Users/Angela/Desktop/Personal/AI_data/melb_data.csv")
raw_melb.shape

(13580, 21)

In [3]:
raw_melb.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [67]:
df = raw_melb.copy()
df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [68]:
df.describe(include = 'object')

,Suburb,Address,Type,Method,SellerG,Date,CouncilArea,Regionname
count,13580,13580,13580,13580,13580,13580,12211,13580
unique,314,13378,3,5,268,58,33,8
top,Reservoir,36 Aberfeldie St,h,S,Nelson,27/05/2017,Moreland,Southern Metropolitan
freq,359,3,9449,9022,1565,473,1163,4695


# 피처, 타깃 데이터 지정

정황상 보고 집의 type을 맞출 수 있을까? 몇가지 컬럼만 가져오도록 합니다.

In [69]:
df = df[['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'Propertycount', 'Regionname', 'Suburb', 'Type', 'Price', 'Distance', 'BuildingArea']]

In [7]:
X = df[['Rooms', 'Price', 'Landsize', 'Propertycount']]
y = df.Type

## 트레이닝, 테스트 데이터 분할

In [8]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 1)

## 데이터 표준화

In [9]:
from sklearn.preprocessing import StandardScaler

std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

# 의사결정나무 분류모델


## 모델링
### 데이터 학습

In [10]:
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier(random_state = 0)
clf_tree.fit(X_tn_std, y_tn)

DecisionTreeClassifier(random_state=0)

### 데이터 예측

In [11]:
tree_pred = clf_tree.predict(X_te_std)
print(tree_pred)

['h' 'h' 'h' ... 'h' 'h' 't']


## 정확도 평가

f1 스코어를 확인해본다.

In [12]:
from sklearn.metrics import f1_score

f1 = f1_score(y_te, tree_pred, average = 'macro')
f1

0.6441165694662292

In [13]:
from sklearn.metrics import recall_score

recall = recall_score(y_te, tree_pred, average = 'macro')
recall

0.6458451226573642

In [14]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_te, tree_pred)
accuracy

0.8144329896907216

정확도 확낮아진걸보니 오기생기고 재미있네요. 애초에 의사결정나무로 집 가격과 소유물 가격들과 해서 집 타입을 맞춘다는게 말도 안되는 것이긴 하지만 저만큼 맞춘게 신기할 정도.


### confusion matrix 확인

confusion matrix를 통해 예측값과 실제값의 일치 정도를 확인해본다.

In [15]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, tree_pred)
conf_matrix

array([[2104,  149,  103],
       [ 124,   77,   71],
       [ 111,   72,  584]])

### 분류 리포트 확인


In [16]:
from sklearn.metrics import classification_report

class_rep = classification_report(y_te, tree_pred)
print(class_rep)

              precision    recall  f1-score   support

           h       0.90      0.89      0.90      2356
           t       0.26      0.28      0.27       272
           u       0.77      0.76      0.77       767

    accuracy                           0.81      3395
   macro avg       0.64      0.65      0.64      3395
weighted avg       0.82      0.81      0.82      3395



## 현재 모형 뜯어보기


### 시각화

In [17]:
# import pydotplus
# from sklearn.tree import export_graphviz
# from IPython.core.display import Image

In [18]:
# dot_data = export_graphviz(clf_tree, out_file=None,
#                            feature_names = X.columns,
#                            class_names = clf_tree.classes_,
#                            filled=True, rounded=True,
#                            special_characters=True)

In [19]:
# dot_data
# graph = pydotplus.graph_from_dot_data(dot_data)
# Image(graph.create_png())

tree 그리는데 문제가 있어서 찾아본 결과:https://moondol-ai.tistory.com/149

에러 내용은 "Graphviz's executables not found" 라고 나오는데요, 이는 환경변수를 등록하지 않아서 발생하는 문제입니다. 

따라서 환경변수에 Graphviz를 추가해주셔야 하는데, 아래의 명령어로 간단하게 수행할 수 있습니다. 

```
import os 
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
```

라고 했지만 맥은 복잡해서 해결을 못하는 중. ㅎ... 일단 다음 실험을 해보자.

# 실험1: 맥스뎁스(10)를 정해주고 만든 모형

이번엔 임의로 트리의 맥스 깊이를 정해주고 만들어본다. 

### 데이터 학습

In [20]:
from sklearn import tree
clf_tree_d10 = tree.DecisionTreeClassifier(random_state = 0, max_depth = 10)
clf_tree_d10.fit(X_tn_std, y_tn)

DecisionTreeClassifier(max_depth=10, random_state=0)

### 데이터 예측

In [21]:
tree_d10_pred = clf_tree_d10.predict(X_te_std)
print(tree_d10_pred)

['h' 'u' 'h' ... 'h' 'h' 'u']


## 정확도 평가

f1 스코어를 확인해본다.

In [22]:
from sklearn.metrics import f1_score

f1 = f1_score(y_te, tree_d10_pred, average = 'macro')
f1

0.64857839066938

In [23]:
from sklearn.metrics import recall_score

recall = recall_score(y_te, tree_d10_pred, average = 'macro')
recall

0.6405346235465386

In [24]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_te, tree_d10_pred)
accuracy

0.8391752577319588

정확도 자체는 조금 더 올라간 것으로 확인되었다.

### confusion matrix 확인

In [25]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, tree_d10_pred)
conf_matrix

array([[2190,   69,   97],
       [ 135,   56,   81],
       [ 121,   43,  603]])

# 실험2: criterion = 'gini' 가 아닌 'entropy'로 했을 때

In [26]:
from sklearn import tree
clf_tree_ent = tree.DecisionTreeClassifier(random_state = 0, criterion = 'entropy')
clf_tree_ent.fit(X_tn_std, y_tn)

DecisionTreeClassifier(criterion='entropy', random_state=0)

### 데이터 예측

In [27]:
tree_ent_pred = clf_tree_ent.predict(X_te_std)
print(tree_ent_pred)

['h' 'u' 'h' ... 'h' 'h' 'h']


## 정확도 평가

f1 스코어를 확인해본다.

In [28]:
from sklearn.metrics import f1_score

f1 = f1_score(y_te, tree_ent_pred, average = 'macro')
f1

0.6411064327841002

In [29]:
from sklearn.metrics import recall_score

recall = recall_score(y_te, tree_ent_pred, average = 'macro')
recall

0.6390807226572427

In [30]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_te, tree_ent_pred)
accuracy

0.8159057437407953

생각보다 정확도가 많이 올라가지 않음을 확인할 수 있었다.

### confusion matrix 확인


In [31]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, tree_ent_pred)
conf_matrix

array([[2121,  127,  108],
       [ 130,   72,   70],
       [ 125,   65,  577]])

```
array([[2104,  149,  103],
       [ 124,   77,   71],
       [ 111,   72,  584]])
```
처음과 비교했을 때 거의 나아진 것이 없다. 

# 실험3: data를 다르게 (Southern Metropolitan 한정)

이번에는 데이터를 수정해보고자 한다. 지역에 따라 타입의 스타일이 다를 수 있기 때문에 한 지역만 한정지어서 해본다.

In [32]:
# df = df[['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'Propertycount', 'Regionname', 'Suburb', 'Type', 'Price']]
df1 = df[df['Regionname'] == 'Southern Metropolitan'][['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'Propertycount', 'Regionname', 'Suburb', 'Type', 'Price']]

In [33]:
X = df1[['Rooms', 'Price', 'Landsize', 'Propertycount']]
y = df1.Type

## 트레이닝, 테스트 데이터 분할

In [34]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 1)

## 데이터 표준화

In [35]:
from sklearn.preprocessing import StandardScaler

std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

## 모델링
### 데이터 학습

In [36]:
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier(random_state = 0)
clf_tree.fit(X_tn_std, y_tn)

DecisionTreeClassifier(random_state=0)

### 데이터 예측

In [37]:
tree_pred = clf_tree.predict(X_te_std)
print(tree_pred)

['h' 't' 'h' ... 'h' 'h' 't']


## 정확도 평가

f1 스코어를 확인해본다.

In [38]:
from sklearn.metrics import f1_score

f1 = f1_score(y_te, tree_pred, average = 'macro')
f1

0.6828771026111623

In [39]:
from sklearn.metrics import recall_score

recall = recall_score(y_te, tree_pred, average = 'macro')
recall

0.6847726282277651

In [40]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_te, tree_pred)
accuracy

0.823679727427598

아주 약간 정확도가 올라간 것을 볼 수 있다. 하지만 만족스러운 정도는 아니다.

### confusion matrix 확인

confusion matrix를 통해 예측값과 실제값의 일치 정도를 확인해본다.

In [41]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, tree_pred)
conf_matrix

array([[610,  47,  38],
       [ 48,  32,  21],
       [ 28,  25, 325]])

### 분류 리포트 확인


In [42]:
from sklearn.metrics import classification_report

class_rep = classification_report(y_te, tree_pred)
print(class_rep)

              precision    recall  f1-score   support

           h       0.89      0.88      0.88       695
           t       0.31      0.32      0.31       101
           u       0.85      0.86      0.85       378

    accuracy                           0.82      1174
   macro avg       0.68      0.68      0.68      1174
weighted avg       0.83      0.82      0.82      1174



아주 약간 나아진 것은 볼 수 있지만, 여전히 좋은 결과는 아닌 것으로 보인다.

그렇다면 피처로 넣은 것들은 집의 타입을 결정하는데 중요한 요소가 아니라는 점일 수 있다.

# 실험4: data를 다르게 (Southern Metropolitan 한정, Bedroom2와 Bathroom 추가)

이번에는 피처를 수정해보고자 한다. 

In [43]:
# df = df[['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'Propertycount', 'Regionname', 'Suburb', 'Type', 'Price']]
df2 = df[(df['Regionname'] == 'Southern Metropolitan')][['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'Propertycount', 'Regionname', 'Suburb', 'Type', 'Price']]

In [44]:
X = df2[['Rooms', 'Price', 'Landsize', 'Propertycount', 'Bedroom2', 'Bathroom']]
y = df2.Type

## 트레이닝, 테스트 데이터 분할

In [45]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 1)

## 데이터 표준화

In [46]:
from sklearn.preprocessing import StandardScaler

std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

## 모델링
### 데이터 학습

In [47]:
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier(random_state = 0)
clf_tree.fit(X_tn_std, y_tn)

DecisionTreeClassifier(random_state=0)

### 데이터 예측

In [48]:
tree_pred = clf_tree.predict(X_te_std)
print(tree_pred)

['h' 'h' 'h' ... 'h' 'h' 'h']


## 정확도 평가

f1 스코어를 확인해본다.

In [49]:
from sklearn.metrics import f1_score

f1 = f1_score(y_te, tree_pred, average = 'macro')
f1

0.7034416404181951

In [50]:
from sklearn.metrics import recall_score

recall = recall_score(y_te, tree_pred, average = 'macro')
recall

0.7043841714307323

In [51]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_te, tree_pred)
accuracy

0.8356047700170358

f1스코어는 상당히 올라간 것을 볼 수 있으며, 정확도 자체는 아주 약간 올라간 것을 볼 수 있다. 

### confusion matrix 확인

confusion matrix를 통해 예측값과 실제값의 일치 정도를 확인해본다.

In [52]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, tree_pred)
conf_matrix

array([[622,  34,  39],
       [ 41,  37,  23],
       [ 26,  30, 322]])

### 분류 리포트 확인


In [53]:
from sklearn.metrics import classification_report

class_rep = classification_report(y_te, tree_pred)
print(class_rep)

              precision    recall  f1-score   support

           h       0.90      0.89      0.90       695
           t       0.37      0.37      0.37       101
           u       0.84      0.85      0.85       378

    accuracy                           0.84      1174
   macro avg       0.70      0.70      0.70      1174
weighted avg       0.84      0.84      0.84      1174



여전히 클래스 t를 분류하는데는 어려움을 겪는 것을 볼 수 있다. 

t값은 어떤 값들로 이루어져 있나?

In [54]:
df2[df2['Type'] == 't']

,Rooms,Bedroom2,Bathroom,Landsize,Propertycount,Regionname,Suburb,Type,Price
86,3,3.0,2.0,718.0,3280.0,Southern Metropolitan,Albert Park,t,1550000.0
224,3,3.0,2.0,101.0,4836.0,Southern Metropolitan,Armadale,t,1205000.0
229,3,3.0,2.0,0.0,4836.0,Southern Metropolitan,Armadale,t,1918000.0
238,3,2.0,2.0,246.0,4836.0,Southern Metropolitan,Armadale,t,1586000.0
274,3,3.0,1.0,103.0,4836.0,Southern Metropolitan,Armadale,t,1101000.0
...,...,...,...,...,...,...,...,...,...
13290,2,2.0,2.0,336.0,5943.0,Southern Metropolitan,South Melbourne,t,817000.0
13343,4,4.0,2.0,338.0,3052.0,Southern Metropolitan,Ashburton,t,1650000.0
13355,4,4.0,2.0,270.0,10969.0,Southern Metropolitan,Bentleigh East,t,1130000.0
13485,4,4.0,2.0,361.0,2397.0,Southern Metropolitan,McKinnon,t,1655000.0


그만 알아보자... 이걸로 상상할 수 있는 부분이 많이 한정적이라니 조금 아쉽다.

# 실험5: 실험4 데이터로 criterion = 'entropy'

혹시나 하는 마음에 계수 결정법을 entropy로 설정해본다.

In [55]:
X = df2[['Rooms', 'Price', 'Landsize', 'Propertycount', 'Bedroom2', 'Bathroom']]
y = df2.Type

## 트레이닝, 테스트 데이터 분할

In [56]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 1)

## 데이터 표준화

In [57]:
from sklearn.preprocessing import StandardScaler

std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

## 모델링
### 데이터 학습

In [58]:
from sklearn import tree
clf_tree_ent = tree.DecisionTreeClassifier(random_state = 0, criterion = 'entropy')
clf_tree_ent.fit(X_tn_std, y_tn)

DecisionTreeClassifier(criterion='entropy', random_state=0)

### 데이터 예측

In [59]:
tree_ent_pred = clf_tree_ent.predict(X_te_std)
print(tree_ent_pred)

['h' 'h' 't' ... 'h' 'h' 'h']


## 정확도 평가

f1 스코어를 확인해본다.

In [60]:
from sklearn.metrics import f1_score

f1 = f1_score(y_te, tree_ent_pred, average = 'macro')
f1

0.712525866026667

In [61]:
from sklearn.metrics import recall_score

recall = recall_score(y_te, tree_ent_pred, average = 'macro')
recall

0.7175437333615742

In [62]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_te, tree_ent_pred)
accuracy

0.8296422487223168

f1스코어는 약간 상승했으나 정확도 자체는 0.01 하락했다.

### confusion matrix 확인

confusion matrix를 통해 예측값과 실제값의 일치 정도를 확인해본다.

In [63]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, tree_ent_pred)
conf_matrix

array([[604,  53,  38],
       [ 46,  42,  13],
       [ 33,  17, 328]])

```
array([[622,  34,  39],
       [ 41,  37,  23],
       [ 26,  30, 322]])
```       
실험4의 결과와 비교해보면 전체적인 정확도가 더 떨어졌음을 한눈에 확인할 수 있다.

### 분류 리포트 확인


In [64]:
from sklearn.metrics import classification_report

class_rep = classification_report(y_te, tree_ent_pred)
print(class_rep)

              precision    recall  f1-score   support

           h       0.88      0.87      0.88       695
           t       0.38      0.42      0.39       101
           u       0.87      0.87      0.87       378

    accuracy                           0.83      1174
   macro avg       0.71      0.72      0.71      1174
weighted avg       0.83      0.83      0.83      1174



더이상 정확도를 올릴 수 있는 방법은 없을 까 고민을 해보긴 하지만, 이 데이터의 한계일 수도 있다는 생각도 든다.

이번엔 디스턴스도 추가해보도록 했다.

# 실험5: data를 다르게 (Southern Metropolitan 한정, Bedroom2와 Bathroom, Distance 추가)

이번에는 피처를 수정해보고자 한다. 

In [70]:
# df = df[['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'Propertycount', 'Regionname', 'Suburb', 'Type', 'Price']]
df2 = df[(df['Regionname'] == 'Southern Metropolitan')][['Rooms', 'Bedroom2', 'Bathroom', 'Landsize', 'Propertycount', 'Regionname', 'Suburb', 'Type', 'Price', 'Distance']]

In [71]:
X = df2[['Rooms', 'Price', 'Landsize', 'Propertycount', 'Bedroom2', 'Bathroom', 'Distance']]
y = df2.Type

## 트레이닝, 테스트 데이터 분할

In [72]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 1)

## 데이터 표준화

In [73]:
from sklearn.preprocessing import StandardScaler

std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

## 모델링
### 데이터 학습

In [74]:
from sklearn import tree
clf_tree = tree.DecisionTreeClassifier(random_state = 0)
clf_tree.fit(X_tn_std, y_tn)

DecisionTreeClassifier(random_state=0)

### 데이터 예측

In [75]:
tree_pred = clf_tree.predict(X_te_std)
print(tree_pred)

['h' 't' 'h' ... 'h' 'h' 't']


## 정확도 평가

f1 스코어를 확인해본다.

In [76]:
from sklearn.metrics import f1_score

f1 = f1_score(y_te, tree_pred, average = 'macro')
f1

0.7100020923023335

In [77]:
from sklearn.metrics import recall_score

recall = recall_score(y_te, tree_pred, average = 'macro')
recall

0.715285222709778

In [78]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_te, tree_pred)
accuracy

0.8313458262350937

별로 좋은 결과가 아닌 것 같다. 현재 크게 영향을 끼치고 있는 다른 요인이 너무 큰 영향력을 끼치고 있는지도 모르겠다.

### confusion matrix 확인

confusion matrix를 통해 예측값과 실제값의 일치 정도를 확인해본다.

In [79]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_te, tree_pred)
conf_matrix

array([[614,  49,  32],
       [ 42,  42,  17],
       [ 32,  26, 320]])

### 분류 리포트 확인


In [80]:
from sklearn.metrics import classification_report

class_rep = classification_report(y_te, tree_pred)
print(class_rep)

              precision    recall  f1-score   support

           h       0.89      0.88      0.89       695
           t       0.36      0.42      0.39       101
           u       0.87      0.85      0.86       378

    accuracy                           0.83      1174
   macro avg       0.71      0.72      0.71      1174
weighted avg       0.84      0.83      0.83      1174



이마저도... 클래스 t를 분류하는데는 어려움을 겪는 것을 볼 수 있다. 
